# dataset path = E:\Mycode\UM_MDSAIA_Assignmentss\CISC7201\final_project\processed_lianjia_data_filtered.csv


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor

# Define features and target variable
df = pd.read_csv('E:\\Mycode\\UM_MDSAIA_Assignmentss\\CISC7201\\final_project\\processed_lianjia_data_filtered.csv')
X = df.drop(columns=['price_per_sqm'])
y = df['price_per_sqm']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the models
dt_regressor = DecisionTreeRegressor(random_state=42)
mlp_regressor = MLPRegressor(random_state=42, max_iter=1000)

# Train the Decision Tree Regressor
dt_regressor.fit(X_train, y_train)
y_pred_dt = dt_regressor.predict(X_test)
mse_dt = mean_squared_error(y_test, y_pred_dt)
print(f'Decision Tree Regressor MSE: {mse_dt}')

# Train the MLP Regressor
mlp_regressor.fit(X_train, y_train)
y_pred_mlp = mlp_regressor.predict(X_test)
mse_mlp = mean_squared_error(y_test, y_pred_mlp)
print(f'MLP Regressor MSE: {mse_mlp}')

Decision Tree Regressor MSE: 3014607.752442997
MLP Regressor MSE: 373817.82477807254


In [11]:
import torch

# 检查是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cpu


In [12]:
# Define features and target variable for price_per_sqm prediction
X_price = df.drop(columns=['price_per_sqm'])
y_price = df['price_per_sqm']

# Split the data into training and testing sets
X_train_price, X_test_price, y_train_price, y_test_price = train_test_split(X_price, y_price, test_size=0.2, random_state=42)

# Initialize the models
dt_regressor_price = DecisionTreeRegressor(random_state=42)
mlp_regressor_price = MLPRegressor(random_state=42, max_iter=1000)

# Train the Decision Tree Regressor
dt_regressor_price.fit(X_train_price, y_train_price)
y_pred_dt_price = dt_regressor_price.predict(X_test_price)
mse_dt_price = mean_squared_error(y_test_price, y_pred_dt_price)
print(f'Decision Tree Regressor MSE for price_per_sqm: {mse_dt_price}')

# Train the MLP Regressor
mlp_regressor_price.fit(X_train_price, y_train_price)
y_pred_mlp_price = mlp_regressor_price.predict(X_test_price)
mse_mlp_price = mean_squared_error(y_test_price, y_pred_mlp_price)
print(f'MLP Regressor MSE for price_per_sqm: {mse_mlp_price}')

Decision Tree Regressor MSE for price_per_sqm: 3014607.752442997
MLP Regressor MSE for price_per_sqm: 373817.82477807254


In [14]:
import torch
import numpy as np

import torch.nn as nn
import torch.optim as optim

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Initialize the MLP model, loss function, and optimizer
mlp_model = MLP(X_train.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(mlp_model.parameters(), lr=0.001)

# Train the MLP model
num_epochs = 1000
for epoch in range(num_epochs):
    mlp_model.train()
    optimizer.zero_grad()
    outputs = mlp_model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

# Evaluate the MLP model
mlp_model.eval()
with torch.no_grad():
    y_pred_mlp_tensor = mlp_model(X_test_tensor)
    mse_mlp = criterion(y_pred_mlp_tensor, y_test_tensor).item()
print(f'Pure MLP Regressor MSE: {mse_mlp}')

# Basic implementation of Random Forest
class RandomForest:
    def __init__(self, n_estimators=100, max_depth=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.trees = [DecisionTreeRegressor(max_depth=self.max_depth) for _ in range(self.n_estimators)]

    def fit(self, X, y):
        for tree in self.trees:
            bootstrap_indices = np.random.choice(np.arange(X.shape[0]), size=X.shape[0], replace=True)
            X_bootstrap = X[bootstrap_indices]
            y_bootstrap = y[bootstrap_indices]
            tree.fit(X_bootstrap, y_bootstrap)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        return np.mean(tree_preds, axis=0)

# Initialize and train the Random Forest model
rf_model = RandomForest(n_estimators=100, max_depth=10)
rf_model.fit(X_train.values, y_train.values)

# Evaluate the Random Forest model
y_pred_rf = rf_model.predict(X_test.values)
mse_rf = mean_squared_error(y_test, y_pred_rf)
print(f'Pure Random Forest Regressor MSE: {mse_rf}')

Pure MLP Regressor MSE: 433872352.0
Pure Random Forest Regressor MSE: 7041528.4241601275
